## Caja en terminal de Jupyter

In [10]:
import csv
from datetime import datetime
import os

class Store:
    def __init__(self):
        self.inventory = {
            "pulsera": [3, 50],
            "sticker": [1, 200],
            "llavero": [10, 12],
            "postit": [3.5, 12],
        }
        self.sales_record = {}

    # Creamos la función display_inventory
    def display_inventory(self):
        print("Lista de Productos:")
        print("{:<15} {:<10} {:<10}".format("Producto", "Precio", "Cantidad"))
        print("-" * 35)

        for producto, info in self.inventory.items():
            precio, cantidad = info
            print("{:<15} {:<10.2f} {:<10}".format(producto, precio, cantidad))

    # Creamos la función purchase_product
    def purchase_product(self):
        try:
            # Solicitar al usuario el nombre del producto
            product_name = input("¿Qué desea comprar?: ").strip().lower()
            # Verificamos si el producto existe en la tienda
            if product_name not in self.inventory:
                print(f"'{product_name}' no está disponible en la tienda.")
                return

            # Solicitar al usuario la cantidad que desea comprar
            quantity = int(input(f"Ingrese la cantidad de '{product_name}' que desea comprar: "))
            if quantity <= 0:
                print("La cantidad debe ser un número positivo.")
                return

            # Obtener el precio y la cantidad disponible del producto
            price, available_quantity = self.inventory[product_name]

            # Verificar si hay suficiente cantidad en inventario
            if quantity <= available_quantity:
                
                # Calcular el precio total
                total_price = price * quantity
                
                # Mostrar el precio total
                print(f"El precio total de {quantity} '{product_name}' es: S/{total_price:.2f}")
                
                # Actualizar la cantidad en inventario
                self.inventory[product_name][1] -= quantity
                
                # Actualizar el registro de ventas
                self.update_sales_record(product_name, quantity)
            else:
                print(f"Lo sentimos, por ahora no contamos con suficiente stock de '{product_name}'.")
                print(f"Stock disponible: {available_quantity} unidades.")
        except ValueError:
            print("Por favor, ingrese una cantidad válida.")
        except Exception as e:
            print(f"Ha ocurrido un error: {e}")

    # Creamos la función update_sales_record
    def update_sales_record(self, producto, cantidad):
        # Obtener la fecha actual (podemos utilizar la biblioteca datetime para esto)
        fecha_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Verificar si la fecha ya existe en el registro de ventas
        if fecha_actual in self.sales_record:
            # Verificar si el producto ya está registrado
            if producto in self.sales_record[fecha_actual]:
                # Actualiza la cantidad vendida del producto
                self.sales_record[fecha_actual][producto] += cantidad
            else:
                # Agrega el producto a la fecha existente con la cantidad correspondiente
                self.sales_record[fecha_actual][producto] = cantidad
        else:
            # Agrega la fecha con el producto y la cantidad correspondientes
            self.sales_record[fecha_actual] = {producto: cantidad}

    # Creamos la función generate_sales_report
    def generate_sales_report(self):
        print("Informe de Ventas:")
        total_store_income = 0 # Inicializa el ingreso total de la tienda

        for fecha_hora, ventas in self.sales_record.items():
            # Separar la fecha y la hora
            fecha, hora = fecha_hora.split()
            print(f"Fecha: {fecha}, Hora: {hora}")
            
            for producto, cantidad in ventas.items():
                # Obtiene el precio del producto desde el inventario
                precio, _ = self.inventory.get(producto, [0, 0]) # Si el producto no existe, se establece el precio en 0
                
                # Calcula los ingresos por producto
                ingresos_producto = precio * cantidad
                total_store_income += ingresos_producto
                print(f"Producto: {producto}, Cantidad Vendida: {cantidad}, Ingresos: S/{ingresos_producto:.2f}")

        # Muestra el ingreso total de la tienda
        print(f"Ingreso Total de la Tienda: S/{total_store_income:.2f}")

    # Creamos la función export_sales_report_to_csv
    def export_sales_report_to_csv(self, filename):
        # Verificar si el nombre del archivo es válido
        if not filename.strip():
            print("El nombre del archivo no puede estar vacío.")
            return

        # Si el nombre no termina en ".csv", se añade al nombre
        if not filename.endswith(".csv"):
            filename += ".csv"

        # Verificar si la ruta del archivo es válida
        directory = os.path.dirname(filename)
        if directory and not os.path.exists(directory):
            print(f"La ruta del archivo '{directory}' no existe.")
            return

        try:
            # Crear o abrir el archivo CSV para escritura
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)

                # Escribir la cabecera del CSV
                writer.writerow(["Fecha", "Hora", "Producto", "Precio", "Cantidad Vendida"])

                # Escribir las filas del CSV
                for fecha_hora, ventas in self.sales_record.items():
                    fecha, hora = fecha_hora.split()
                    for producto, cantidad_vendida in ventas.items():
                        # Obtener el precio inicial del producto desde el inventario
                        precio, _ = self.inventory.get(producto, [0, 0])
                        writer.writerow([fecha, hora, producto, precio, cantidad_vendida])

            print(f"El informe de ventas ha sido exportado a {filename}")
        except Exception as e:
            print(f"Ha ocurrido un error al exportar el informe: {e}")

    # Creamos la función export_inventory_to_csv
    def export_inventory_to_csv(self, filename):
        if not filename.strip():
            print("El nombre del archivo no puede estar vacío.")
            return

        if not filename.endswith(".csv"):
            filename += ".csv"

        directory = os.path.dirname(filename)
        if directory and not os.path.exists(directory):
            print(f"La ruta del archivo '{directory}' no existe.")
            return

        try:
            # Crear o abrir el archivo CSV para escritura
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                
                # Escribir la cabecera del CSV
                writer.writerow(["Producto", "Precio", "Cantidad"])

                # Escribir las filas del CSV
                for producto, (precio, cantidad) in self.inventory.items():
                    writer.writerow([producto, precio, cantidad])

            print(f"El inventario ha sido exportado a {filename}")
        except Exception as e:
            print(f"Ha ocurrido un error al exportar el inventario: {e}")

    # Creamos la función load_inventory_from_csv
    def load_inventory_from_csv(self, filename):
        if not os.path.exists(filename):
            print(f"El archivo '{filename}' no existe.")
            return

        try:
            # Abrir el archivo CSV para leer
            with open(filename, mode='r', newline='') as file:
                reader = csv.reader(file)
                next(reader)  # Saltar la cabecera
                self.inventory = {rows[0]: [float(rows[1]), int(rows[2])] for rows in reader}

            print(f"Inventario cargado desde '{filename}'")
        except Exception as e:
            print(f"Ha ocurrido un error al cargar el inventario: {e}")

def main():
    store = Store()

    while True:
        # Paso 1: Muestra el menú de opciones
        print("\nMenú de Opciones:")
        print("1. Mostrar Inventario")
        print("2. Comprar Producto")
        print("3. Generar Informe de Ventas")
        print("4. Exportar Informe de Ventas a CSV")
        print("5. Exportar Inventario a CSV")
        print("6. Cargar Inventario desde CSV")
        print("7. Salir")

        # Paso 2: Solicitar al usuario que elija una opción
        opcion = input("Elija una opción (1/2/3/4/5/6/7): ")

        # Paso 3: Implementar las opciones del menú
        if opcion == "1":
            # Opción: Mostrar Inventario
            store.display_inventory()
        elif opcion == "2":
            # Opción: Comprar Producto
            store.purchase_product()
        elif opcion == "3":
            # Opción: Generar Informe de Ventas
            store.generate_sales_report()
        elif opcion == "4":
            # Opción: Exportar Informe de Ventas a CSV
            filename = input("Ingrese el nombre del archivo CSV (con extensión .csv): ").strip()
            store.export_sales_report_to_csv(filename)
        elif opcion == "5":
            # Opción: Exportar Inventario a CSV
            filename = input("Ingrese el nombre del archivo CSV del inventario (con extensión .csv): ").strip()
            store.export_inventory_to_csv(filename)
        elif opcion == "6":
            # Opción: Cargar Inventario desde CSV
            filename = input("Ingrese el nombre del archivo CSV del inventario (con extensión .csv): ").strip()
            store.load_inventory_from_csv(filename)
        elif opcion == "7":
            # Opción: Salir
            print("Saliendo del programa.")
            break
        else:
            print("Por favor, elija una opción válida.")

if __name__ == "__main__":
    main()



Menú de Opciones:
1. Mostrar Inventario
2. Comprar Producto
3. Generar Informe de Ventas
4. Exportar Informe de Ventas a CSV
5. Exportar Inventario a CSV
6. Cargar Inventario desde CSV
7. Salir
Lista de Productos:
Producto        Precio     Cantidad  
-----------------------------------
pulsera         3.00       50        
sticker         1.00       200       
llavero         10.00      12        
postit          3.50       12        

Menú de Opciones:
1. Mostrar Inventario
2. Comprar Producto
3. Generar Informe de Ventas
4. Exportar Informe de Ventas a CSV
5. Exportar Inventario a CSV
6. Cargar Inventario desde CSV
7. Salir
El precio total de 11 'pulsera' es: S/33.00

Menú de Opciones:
1. Mostrar Inventario
2. Comprar Producto
3. Generar Informe de Ventas
4. Exportar Informe de Ventas a CSV
5. Exportar Inventario a CSV
6. Cargar Inventario desde CSV
7. Salir
Lista de Productos:
Producto        Precio     Cantidad  
-----------------------------------
pulsera         3.00       39  

## Caja GUI usando 'tkinter'

In [9]:
import csv
from datetime import datetime
import os
import tkinter as tk
from tkinter import messagebox, filedialog

class Store:
    def __init__(self):
        # Inicializa el inventario con productos y sus precios y cantidades
        self.inventory = {
            "pulsera": [3, 50],
            "sticker": [1, 200],
            "llavero": [10, 12],
            "postit": [3.5, 12],
        }
        # Inicializa el registro de ventas
        self.sales_record = {}

    def display_inventory(self):
        # Crea una lista con el encabezado de la tabla
        inventory_list = ["{:<15} {:<10} {:<10}".format("Producto", "Precio", "Cantidad")]
        # Añade cada producto y su información formateada
        for producto, info in self.inventory.items():
            precio, cantidad = info
            inventory_list.append("{:<15} {:<10.2f} {:<10}".format(producto, precio, cantidad))
        # Devuelve el inventario como una cadena de texto
        return "\n".join(inventory_list)

    def purchase_product(self, product_name, quantity):
        # Verifica si el producto está en el inventario
        if product_name not in self.inventory:
            return f"'{product_name}' no está disponible en la tienda."

        # Verifica si la cantidad es válida
        if quantity <= 0:
            return "La cantidad debe ser un número positivo."

        # Obtiene el precio y la cantidad disponible del producto
        price, available_quantity = self.inventory[product_name]

        # Verifica si hay suficiente stock
        if quantity <= available_quantity:
            # Calcula el precio total y actualiza el inventario
            total_price = price * quantity
            self.inventory[product_name][1] -= quantity
            # Actualiza el registro de ventas
            self.update_sales_record(product_name, quantity)
            return f"El precio total de {quantity} '{product_name}' es: S/{total_price:.2f}"
        else:
            return f"Lo sentimos, por ahora no contamos con suficiente stock de '{product_name}'. Stock disponible: {available_quantity} unidades."

    def update_sales_record(self, producto, cantidad):
        # Obtiene la fecha y hora actual
        fecha_hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        fecha_actual = fecha_hora_actual.split()[0]
        hora_actual = fecha_hora_actual.split()[1]

        # Añade la venta al registro, con fecha, hora y cantidad
        if fecha_actual in self.sales_record:
            self.sales_record[fecha_actual].append((hora_actual, producto, cantidad))
        else:
            self.sales_record[fecha_actual] = [(hora_actual, producto, cantidad)]

    def generate_sales_report(self):
        # Inicializa la lista del informe con un encabezado
        report_list = ["Informe de Ventas:"]
        total_store_income = 0

        # Recorre cada fecha en el registro de ventas
        for fecha, ventas in self.sales_record.items():
            report_list.append(f"Fecha: {fecha}")
            # Recorre cada venta en la fecha
            for venta in ventas:
                hora, producto, cantidad = venta
                precio, _ = self.inventory.get(producto, [0, 0])
                ingresos_producto = precio * cantidad
                total_store_income += ingresos_producto
                # Añade la información de cada venta al informe
                report_list.append(f"Hora: {hora}, Producto: {producto}, Cantidad Vendida: {cantidad}, Ingresos: S/{ingresos_producto:.2f}")
        
        # Añade el ingreso total al final del informe
        report_list.append(f"Ingreso Total de la Tienda: S/{total_store_income:.2f}")
        return "\n".join(report_list)

    def export_sales_report_to_csv(self, filename):
        # Verifica que el nombre del archivo no esté vacío y que termine en .csv
        if not filename.strip():
            return "El nombre del archivo no puede estar vacío."
        
        if not filename.endswith(".csv"):
            filename += ".csv"
        
        directory = os.path.dirname(filename)
        if directory and not os.path.exists(directory):
            return f"La ruta del archivo '{directory}' no existe."
        
        try:
            # Crea o abre el archivo CSV para escritura
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Fecha", "Hora", "Producto", "Precio", "Cantidad Vendida"])
                
                # Escribe cada registro de ventas en el archivo CSV
                for fecha, ventas in self.sales_record.items():
                    for venta in ventas:
                        hora, producto, cantidad_vendida = venta
                        precio, _ = self.inventory.get(producto, [0, 0])
                        writer.writerow([fecha, hora, producto, precio, cantidad_vendida])
            
            return f"El informe de ventas ha sido exportado a {filename}"
        except Exception as e:
            return f"Ha ocurrido un error al exportar el informe: {e}"

    def export_inventory_to_csv(self, filename):
        # Verifica que el nombre del archivo no esté vacío y que termine en .csv
        if not filename.strip():
            return "El nombre del archivo no puede estar vacío."
        
        if not filename.endswith(".csv"):
            filename += ".csv"
        
        directory = os.path.dirname(filename)
        if directory and not os.path.exists(directory):
            return f"La ruta del archivo '{directory}' no existe."
        
        try:
            # Crea o abre el archivo CSV para escritura
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Producto", "Precio", "Cantidad"])
                
                # Escribe cada producto en el archivo CSV
                for producto, (precio, cantidad) in self.inventory.items():
                    writer.writerow([producto, precio, cantidad])
            
            return f"El inventario ha sido exportado a {filename}"
        except Exception as e:
            return f"Ha ocurrido un error al exportar el inventario: {e}"

    def load_inventory_from_csv(self, filename):
        # Verifica que el archivo exista
        if not os.path.exists(filename):
            return f"El archivo '{filename}' no existe."
        
        try:
            # Abre el archivo CSV para lectura
            with open(filename, mode='r', newline='') as file:
                reader = csv.reader(file)
                next(reader)  # Saltar la cabecera
                self.inventory = {rows[0]: [float(rows[1]), int(rows[2])] for rows in reader}
            
            return f"Inventario cargado desde '{filename}'"
        except Exception as e:
            return f"Ha ocurrido un error al cargar el inventario: {e}"

class StoreGUI:
    def __init__(self, root):
        # Inicializa la tienda y la ventana raíz de tkinter
        self.store = Store()
        self.root = root
        self.root.title("Simulador de Caja de Venta")

        # Crea los elementos de la interfaz
        self.create_widgets()

    def create_widgets(self):
        # Botón para mostrar el inventario
        self.inventory_button = tk.Button(self.root, text="Mostrar Inventario", command=self.show_inventory)
        self.inventory_button.grid(row=0, column=0, padx=10, pady=10)

        # Etiqueta y campos de entrada para comprar productos
        self.purchase_label = tk.Label(self.root, text="Comprar Producto:")
        self.purchase_label.grid(row=1, column=0, padx=10, pady=10)

        self.product_label = tk.Label(self.root, text="Producto:")
        self.product_label.grid(row=2, column=0, padx=10, pady=10)
        self.product_entry = tk.Entry(self.root)
        self.product_entry.grid(row=2, column=1, padx=10, pady=10)

        self.quantity_label = tk.Label(self.root, text="Cantidad:")
        self.quantity_label.grid(row=3, column=0, padx=10, pady=10)
        self.quantity_entry = tk.Entry(self.root)
        self.quantity_entry.grid(row=3, column=1, padx=10, pady=10)

        # Botón para realizar la compra
        self.purchase_button = tk.Button(self.root, text="Comprar", command=self.purchase_product)
        self.purchase_button.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        # Botones para generar y exportar informes
        self.report_button = tk.Button(self.root, text="Generar Informe de Ventas", command=self.generate_sales_report)
        self.report_button.grid(row=5, column=0, columnspan=2, padx=10, pady=10)

        self.export_sales_button = tk.Button(self.root, text="Exportar Informe de Ventas a CSV", command=self.export_sales_report)
        self.export_sales_button.grid(row=6, column=0, columnspan=2, padx=10, pady=10)

        self.export_inventory_button = tk.Button(self.root, text="Exportar Inventario a CSV", command=self.export_inventory)
        self.export_inventory_button.grid(row=7, column=0, columnspan=2, padx=10, pady=10)

        # Botón para cargar el inventario desde un archivo CSV
        self.load_inventory_button = tk.Button(self.root, text="Cargar Inventario desde CSV", command=self.load_inventory)
        self.load_inventory_button.grid(row=8, column=0, columnspan=2, padx=10, pady=10)

    def show_inventory(self):
        # Muestra el inventario en un cuadro de mensaje
        inventory = self.store.display_inventory()
        messagebox.showinfo("Inventario", inventory)

    def purchase_product(self):
        # Realiza una compra y muestra el resultado en un cuadro de mensaje
        product_name = self.product_entry.get()
        try:
            quantity = int(self.quantity_entry.get())
        except ValueError:
            messagebox.showerror("Error", "La cantidad debe ser un número entero.")
            return
        
        result = self.store.purchase_product(product_name, quantity)
        messagebox.showinfo("Compra", result)

    def generate_sales_report(self):
        # Genera un informe de ventas y lo muestra en un cuadro de mensaje
        report = self.store.generate_sales_report()
        messagebox.showinfo("Informe de Ventas", report)

    def export_sales_report(self):
        # Exporta el informe de ventas a un archivo CSV
        filename = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if filename:
            result = self.store.export_sales_report_to_csv(filename)
            messagebox.showinfo("Exportar Informe de Ventas", result)

    def export_inventory(self):
        # Exporta el inventario a un archivo CSV
        filename = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if filename:
            result = self.store.export_inventory_to_csv(filename)
            messagebox.showinfo("Exportar Inventario", result)

    def load_inventory(self):
        # Carga el inventario desde un archivo CSV
        filename = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if filename:
            result = self.store.load_inventory_from_csv(filename)
            messagebox.showinfo("Cargar Inventario", result)

if __name__ == "__main__":
    root = tk.Tk()
    app = StoreGUI(root)
    root.mainloop()
